 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

### 1. Import libraries and dependencies

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns
%matplotlib inline

### 2. Create a Path to the File Using Pathlib

In [ ]:
# Use the Pathlib libary to set the path to the CSV
whale_returns_csv = Path("Resources/whale_returns.csv")
algo_returns_csv = Path("Resources/algo_returns.csv")
sptsx60_history_csv = Path("Resources/sp_tsx_history.csv")

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_returns_df = pd.read_csv(whale_returns_csv,index_col='Date', infer_datetime_format=True, parse_dates = True)
whale_returns_df.head()

In [ ]:
# Count nulls
whale_returns_df.isnull().sum()

In [ ]:
# Determine percentage of nulls
whale_returns_df.isnull().mean() * 100

In [ ]:
# Drop nulls
whale_returns_df.dropna(inplace=True)
whale_returns_df.isnull().sum()
whale_returns_df.head()

In [ ]:
# Check count nulls
whale_returns_df.isnull().sum()

In [ ]:
# Check determine percentage of nulls
whale_returns_df.isnull().mean() * 100

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_df = pd.read_csv(algo_returns_csv, index_col='Date', infer_datetime_format=True, parse_dates = True)
algo_returns_df.head(10)

In [ ]:
# Count nulls
algo_returns_df.isnull().sum()

In [ ]:
# Drop nulls
algo_returns_df.dropna(inplace=True)
algo_returns_df.head()

In [ ]:
# Check count nulls
algo_returns_df.isnull().sum()

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P TSX 60 Closing Prices
sptsx60_history_df = pd.read_csv(sptsx60_history_csv, index_col='Date', infer_datetime_format=True, parse_dates = True)
sptsx60_history_df.head()

In [ ]:
# Check Data Types
sptsx60_history_df.dtypes                                                                                                                                                               

In [ ]:
# Fix Data Types
sptsx60_history_df["Close"] = sptsx60_history_df["Close"].str.replace('$','',regex=False)
sptsx60_history_df.head()

In [ ]:
# Fix Data Types
sptsx60_history_df["Close"] = sptsx60_history_df["Close"].str.replace(',','',regex=False)
sptsx60_history_df.head()

In [ ]:
# sp500_history.to_numeric
sptsx60_history_df['Close'] = sptsx60_history_df['Close'].astype(float)
sptsx60_history_df.dtypes

In [ ]:
# Calculate Daily Returns
sptsx60_history_df = sptsx60_history_df.pct_change()
sptsx60_history_df.head()

In [ ]:
# Drop nulls
sptsx60_history_df.dropna(inplace=True)
sptsx60_history_df.head()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sptsx60_history_df.rename(columns={"Close": "S&P TSX"},inplace = True)
sptsx60_history_df.head()

## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combined_whale_algo_sptsx60_df = pd.concat([whale_returns_df, algo_returns_df, sptsx60_history_df], axis="columns", join="inner")
combined_whale_algo_sptsx60_df.head()

In [ ]:
combined_whale_algo_sptsx60_df.tail()

In [ ]:
combined_whale_algo_sptsx60_df.describe()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_whale_algo_sptsx60_df.plot(figsize=(20,10), title = "Daily Returns")

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 +combined_whale_algo_sptsx60_df).cumprod()

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
combined_whale_algo_sptsx60_df_boxplot = combined_whale_algo_sptsx60_df.boxplot(figsize = (20,10), grid = False)

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
combined_whale_algo_sptsx60_df_std = combined_whale_algo_sptsx60_df.std()
combined_whale_algo_sptsx60_df_std.sort_values(inplace=True)
combined_whale_algo_sptsx60_df_std

### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate  the daily standard deviation of S&P TSX 60
combined_whale_algo_sptsx60_df['S&P TSX']

In [ ]:
# Determine which portfolios are riskier than the S&P TSX 60
combined_whale_algo_sptsx60_df_std > combined_whale_algo_sptsx60_df_std['S&P TSX']

In [ ]:
combined_whale_algo_sptsx60_df_std > combined_whale_algo_sptsx60_df_std['Algo 2']

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annual_std = combined_whale_algo_sptsx60_df.std() * np.sqrt(252)
annual_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
group_rolling = combined_whale_algo_sptsx60_df.rolling(window=21).std()
group_rolling

In [ ]:
# Plot the rolling standard deviation
group_rolling = combined_whale_algo_sptsx60_df.rolling(window=21).std().plot(figsize = (20,10))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
corr_returns = combined_whale_algo_sptsx60_df.corr()
corr_returns

In [ ]:
# Display de correlation matrix
sns.heatmap(corr_returns, vmin = -1, vmax = 1, annot=True, cmap="Blues")

### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculate covariance of a single portfolio
berkshire_covariance = whale_returns_df['BERKSHIRE HATHAWAY INC'].cov
berkshire_covariance

In [ ]:
# Calculate variance of S&P TSX
spTSX_variance = combined_whale_algo_sptsx60_df['S&P TSX'].rolling(window=60).var()
spTSX_variance

In [ ]:
# Computing beta
berkshire_spTSX = combined_whale_algo_sptsx60_df['BERKSHIRE HATHAWAY INC'].rolling(window=60).cov(combined_whale_algo_sptsx60_df['S&P TSX'])
berkshire_spTSX

In [ ]:
berkshire_beta = berkshire_spTSX / spTSX_variance
berkshire_beta

In [ ]:
# Plot beta trend
berkshire_beta.plot(figsize = (20,10))

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window
ewm = combined_whale_algo_sptsx60_df.ewm(halflife = 21).mean().plot(figsize=(20,10))

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
annual_sharpe_ratios = (combined_whale_algo_sptsx60_df.mean() * 252) / (combined_whale_algo_sptsx60_df.std() * np.sqrt(252))
annual_sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
annual_sharpe_ratios.plot(kind = 'bar')

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

Write your answer here!


short answer yes Algorithmic 1 & 2 (Algo 1, Algo 2)out performed both 'S&P TSX 60' and 'whales porfolios', when looking at portfolio to portfolio as a whole. Mainly Algo 1 was able to generate higher returns on a risk managed basis.

**Combined Return:** Algo 1 & 2 are unexeceptional compared to Whales & S&P TSX 60 portfolio. Overall figures in 'describ()' Berkshire stock from whales portfolio dtands out to be the better return, however the remaining whales stock evens the out performing Berkshire stock.

**Daily returns Plot:** Tiger global stock from whale returns displays extreme highs and lows indicating high volatility low cofidence in stock. S&P TSX appears consistent. Berkshire appears to be the better perfoming stock with beter returns but also same lows.

**Cumulative Returns PLot:** Algo 1 outperforms all portfolios. consistent growth, not much volatility action.

**Box Plot:** Can be seen as a summary of previous data Algo 1 has more cluster in the 95%tile, Tiger stock has the highest volatility showing the highest and lowest point indicator. Berkshire altho has the most consistent data to the mean, this stock also has more data in the 5%tile

**Standard Deviation:** all unremarkable to each other. All stock are relatively close to mean

**Standard Deviation Plot:** Berkshire & Tiger stocks standout showing higher deviation from the mean. Algo 1 & 2 are relatively consistent to the mean

**Correlation:** Algo 2 is the better combination to other stocks, oddely enough being less compatible to Algo 1. Whales portfolio consists of stock that are compatiable to each other.

**Sharpe Ratio:** Algo 1 was able to generate higher returns on a risk managed basis as indicated by the higher sharpe ratio value




# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
aveo_historical_csv = Path("Resources/aveo_historical.csv")
aveo_historical_df = pd.read_csv(aveo_historical_csv, index_col='Date', infer_datetime_format=True, parse_dates = True)
aveo_historical_df

In [ ]:
# Reading data from 2nd stock
cldx_historical_csv = Path("Resources/cldx_historical.csv")
cldx_historical_df = pd.read_csv(cldx_historical_csv, index_col='Date', infer_datetime_format=True, parse_dates = True)
cldx_historical_df.head()

In [ ]:
# Reading data from 3rd stock
ptc_historical_csv = Path("Resources/ptc_historical.csv")
ptc_historical_df = pd.read_csv(ptc_historical_csv, index_col='Date', infer_datetime_format=True, parse_dates = True)
ptc_historical_df.head()

In [ ]:
# Combine all stocks in a single DataFrame
combine_custom_stocks = pd.concat([aveo_historical_df, cldx_historical_df, ptc_historical_df], axis = 1, join = "inner")
combine_custom_stocks.sort_index(inplace=True)
combine_custom_stocks.head()

In [ ]:
combine_custom_stocks.index = combine_custom_stocks.index.normalize()
combine_custom_stocks.head()

In [ ]:
# Reset Date index
combine_custom_stocks.sort_index(inplace=True)



In [ ]:
# Reorganize portfolio data by having a column per symbol
combine_custom_stocks.columns = ['AVEO', 'CLDX', 'PTC']
#combine_custom_stocks.drop(index=0, inplace=True)
combine_custom_stocks.head()


In [ ]:
#combine_custom_stocks.columns
#print(combine_custom_stocks.iloc[0, :])
#combine_custom_stocks

In [ ]:
# Calculate daily returns
custom_main = combine_custom_stocks.pct_change()
custom_main.head()

In [ ]:
# Drop NAs
combine_custom_stocks.dropna(inplace=True)
combine_custom_stocks.head()
# Display sample data

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
portfolio_returns = custom_main.dot(weights)

# Display sample data

portfolio_returns.isnull().mean() * 100
portfolio_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
join_custom_combined_df= pd.concat([combined_whale_algo_sptsx60_df, portfolio_returns], axis =1, join= "inner")
join_custom_combined_df.rename(columns={0: 'Custom'},inplace=True)
join_custom_combined_df.head()

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
join_custom_combined_df.isnull().sum()
mod_df = join_custom_combined_df.drop(join_custom_combined_df.index[0],inplace=True)
join_custom_combined_df.head()

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
join_annual_std = join_custom_combined_df.std() * np.sqrt(252)
join_annual_std

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
join_custom_combined_df.rolling(window=21).std()

In [ ]:
# Plot rolling standard deviation
join_custom_combined_df.rolling(window=21).std().plot(figsize =(20,10) )

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
custom_corr_returns = join_custom_combined_df.corr()
custom_corr_returns

### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta
custom_sptsx = join_custom_combined_df['Custom'].rolling(window=60).cov(join_custom_combined_df['S&P TSX'])
custom_sptsx_variance = join_custom_combined_df['S&P TSX'].rolling(window=60).var()
custom_beta = custom_sptsx / custom_sptsx_variance
custom_beta.plot(figsize = (20,10))

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios
custom_annual_sharpe_ratios = (join_custom_combined_df.mean() * 252) / (join_custom_combined_df.std() * np.sqrt(252))
custom_annual_sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
custom_annual_sharpe_ratios.plot(kind = 'bar')

### How does your portfolio do?

Write your answer here!


My custom portfolio did not perform well at all. not compared to whale, algo 1 & 2, and S&PTSX. The volatile value in custom stock is far to high indicating low confidence in stock. Standard deviation is also to high compared to the other portfolios meaning too many data are far from mean, non consistent data.